Импорт библиотек

In [210]:
import pandas as pd
import numpy as np

Загрузка данных

In [211]:
Sales = pd.read_excel('data\Загрузка продажи.xlsx')
Stocks = pd.read_excel('data\Остатки загрузка.xlsx')
Nomen = pd.read_excel('data\Загрузка справочник.xlsx')

In [212]:
Sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3970 entries, 0 to 3969
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ДАТА        3970 non-null   object 
 1   АРТИКУЛ     3970 non-null   int64  
 2   МЕНЕДЖЕР    3970 non-null   object 
 3   КЛИЕНТ      3970 non-null   object 
 4   ПРОДАЖА ШТ  3970 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 155.2+ KB


In [213]:
Sales.head(2)

,ДАТА,АРТИКУЛ,МЕНЕДЖЕР,КЛИЕНТ,ПРОДАЖА ШТ
0,06.03.2023 г.,101218,Абзалова Ольга Валерьевна,ЧЛ Талгат Айна базар,1.0
1,06.03.2023 г.,101459,Черноиванова Марина,ТОО Magnum Cash&Carry,1200.0


In [214]:
Sales['ДАТА'] = Sales['ДАТА'].apply(lambda x: x[:-3])
Sales['ДАТА'] = pd.to_datetime(Sales['ДАТА'])
Sales.head(2)

C:\Users\killr\AppData\Local\Temp\ipykernel_10416\914593808.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  Sales['ДАТА'] = pd.to_datetime(Sales['ДАТА'])


,ДАТА,АРТИКУЛ,МЕНЕДЖЕР,КЛИЕНТ,ПРОДАЖА ШТ
0,2023-06-03,101218,Абзалова Ольга Валерьевна,ЧЛ Талгат Айна базар,1.0
1,2023-06-03,101459,Черноиванова Марина,ТОО Magnum Cash&Carry,1200.0


In [215]:
Stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21119 entries, 0 to 21118
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ДАТА     21119 non-null  object 
 1   АРТИКУЛ  21119 non-null  int64  
 2   РАСХОД   21119 non-null  float64
 3   ОСТАТОК  21119 non-null  float64
 4   ЦЕНА     21119 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 825.1+ KB


In [216]:
Stocks['ДАТА'] = pd.to_datetime(Stocks['ДАТА'])
Stocks.head(2)

C:\Users\killr\AppData\Local\Temp\ipykernel_10416\1352275529.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  Stocks['ДАТА'] = pd.to_datetime(Stocks['ДАТА'])


,ДАТА,АРТИКУЛ,РАСХОД,ОСТАТОК,ЦЕНА
0,2023-06-03,100160,0.0,23495.0,3.06
1,2023-06-03,100527,0.0,3800.0,43.70


In [217]:
Nomen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   АРТИКУЛ     456 non-null    int64 
 1   ТОВАР       456 non-null    object
 2   ПРЯМОЙ      456 non-null    int64 
 3   ПОСТАВЩИК   456 non-null    object
 4   АКТИВНОСТЬ  456 non-null    object
 5   АГ          456 non-null    object
 6   НГ          456 non-null    object
dtypes: int64(2), object(5)
memory usage: 25.1+ KB


Создадим таблицу по параметрам артикулов и добавим в нее параметры, связанные с клиентами и статистикой продаж

*Число клиентов*

In [218]:
table = Sales.groupby('АРТИКУЛ')['КЛИЕНТ'].nunique()

In [219]:
Client_count = list(Nomen['АРТИКУЛ'].unique())
Client_count = pd.DataFrame(Client_count, columns=['АРТИКУЛ'])
Client_count['Count'] = Client_count['АРТИКУЛ'].apply(lambda x: 0 if x not in list(table.index) else table[x])
Client_count.index = Client_count['АРТИКУЛ']
Client_count = Client_count.drop('АРТИКУЛ', axis=1)

In [220]:
piv_nom = Nomen.copy()
piv_nom['КЛИЕНТОВ'] = piv_nom['АРТИКУЛ'].apply(lambda x: Client_count.loc[x])

*Доля самого крупного клиента*

In [221]:
table = Sales.groupby(['АРТИКУЛ', 'КЛИЕНТ'])['ПРОДАЖА ШТ'].sum().reset_index().sort_values(by=['АРТИКУЛ','ПРОДАЖА ШТ'], ascending=False)
table2 = Sales.groupby(['АРТИКУЛ'])['ПРОДАЖА ШТ'].sum().sort_values(ascending=False).reset_index()
table = pd.merge(left=table, right=table2, on='АРТИКУЛ', how='left')
table['ДОЛЯ'] = table['ПРОДАЖА ШТ_x'] / table['ПРОДАЖА ШТ_y']
table = table.groupby('АРТИКУЛ')['ДОЛЯ'].max()
piv_nom['ДОЛЯ МАКС КЛИЕНТА'] = piv_nom['АРТИКУЛ'].apply(lambda x: 0 if x not in list(table.index) else table.loc[x])
piv_nom.head(2)

,АРТИКУЛ,ТОВАР,ПРЯМОЙ,ПОСТАВЩИК,АКТИВНОСТЬ,АГ,НГ,КЛИЕНТОВ,ДОЛЯ МАКС КЛИЕНТА
0,100084,"Перчатки одноразовые ""Elpi"", размер M, 6 мкм, ...",0,ЭЛПИ-ТРЕЙД ООО,Да,Средства индивидуальной защиты,Перчатки ПЭ стандартной прочности размер M,5,0.598404
1,100160,"Стакан одноразовый 100 мл ""Сп"", прозр., ПП, 10...",0,ПолиЭр ООО,Да,"Стаканы, чашки, крышки для стаканов",Стаканы ПП до 200 мл прозрачные,8,0.258148


*Загрузка средней цены за период*